In [ ]:
import logging
from typing import Union

import numba
import numpy as np
from numpy.typing import ArrayLike
from pandas import DataFrame, Series

logger = logging.getLogger(__name__)

Approximate GCD:


$$\begin{aligned}
𝗀𝖼𝖽_ϵ(x) 
= 𝗆𝖺𝗑\{y∣ ∀i : 𝖽𝗂𝗌𝗍(x_i, yℤ)≤ϵ\} 
= 𝗆𝖺𝗑\{y∣ ∀i : \min_{k\in\mathbb Z}: |x_i- k y|≤ϵ\} 
\end{aligned}$$



$$\begin{aligned}
𝗀𝖼𝖽_{ϵ,\delta}(x) = 𝗆𝖺𝗑\{y∣ ∀i:\min_k |x_i - k y| ≤ \delta |x_i| + ϵ\}
\end{aligned}$$



In [ ]:
def float_gcd(a, b, rtol=0, atol=1e-02):
    t = min(abs(a), abs(b))
    while abs(b) > rtol * t + atol:
        a, b = b, a % b
    return a

In [ ]:
a, b, c = 1.2, 1.4, 1.59999

In [ ]:
eps = 2**-7
float_gcd(1, 2 + eps * 2, atol=eps / 16)

In [ ]:
2 + eps * 2

In [ ]:
float_gcd(a, float_gcd(b, c)), float_gcd(b, float_gcd(a, c)), float_gcd(
    c, float_gcd(a, b)
),

In [ ]:
float_gcd(a, float_gcd(c, b)), float_gcd(b, float_gcd(c, a)), float_gcd(
    c, float_gcd(b, a)
),

In [ ]:
float_gcd(a, float_gcd(b, c)), float_gcd(b, float_gcd(a, c)), float_gcd(
    c, float_gcd(a, b)
),

In [ ]:
find set(k| xi - ky ) < eps

In [ ]:
xi > ky => xi - ky < eps =>  k > (xi - eps)/y
xi < ky => -xi + ky < eps => k < (xi + eps)/y


min_k |xi - ky| => k = floor(xi/y) or k = ceil(xi/y), prolly k = round(xi/y)

| xi - round(xi/y) y| < eps

In [ ]:
increase y until round flips....

In [ ]:
desiderata: scale invariance: eps-gcd(a*x) = a*eps-gcd(x) 

In [ ]:
1.3, 1.7 



eps > 0.2 => 1.5
eps < 0.2

In [ ]:
atol rtol

|xhat - x| <= rtol*|x| + atol


In [ ]:
def approx_float_gcd(x: ArrayLike, rtol: float = 1e-05, atol: float = 1e-08) -> float:
    r"""Compute approximate GCD of multiple floats.

    .. math::
        𝗀𝖼𝖽_ϵ(x) = 𝗆𝖺𝗑\{y∣ ∀i : 𝖽𝗂𝗌𝗍(x_i, yℤ)≤ϵ\}

    .. warning::
       This implementation does not work 100% correctly yet!

    Parameters
    ----------
    x: ArrayLike
    rtol: float
    atol: float

    Returns
    -------
    float

    References
    ----------
    - <https://stackoverflow.com/q/45323619/9318372>
    """
    logger.warning(
        "The implementation of approx_float_gcd does not work 100% correctly yet!"
    )
    x = np.asanyarray(x)
    x = np.abs(x).flatten()

    @numba.njit
    def _float_gcd(x: np.ndarray) -> float:
        n = len(x)
        t = np.min(x)
        if n == 1:
            return float(x[0])
        if n == 2:
            while np.abs(x[1]) > rtol * t + atol:
                x[0], x[1] = x[1], x[0] % x[1]
            return float(x[0])
        # n >= 3:
        out = np.empty(2)
        out[0] = _float_gcd(x[: n // 2])
        out[1] = _float_gcd(x[n // 2 :])
        return _float_gcd(out)

    return _float_gcd(x)

In [ ]:
approx_float_gcd([0, 1, 1.99], rtol=0)